In [ ]:
PHI_tool = {
    "name": "PHI",
    "description": "Access or create a protected health information (PHI) record for a patient.",
    "input_schema": {
        "type": "object",
        "properties": {
            "intent": {
                "type": "string",
                "enum": ["Create PHI", "Access PHI"],
                "description": "The intent of the PHI related request."
            },
            "full_name": {
                "type": "string",
                "description": "The full name of the patient."
            },
            "DOB": {
                "type": "string",
                "format": "date",
                "description": "The date of birth of the patient."
            },
            "phone_number": {
                "type": "string",
                "description": "The primary phone number of the patient."
            },
            "address": {
                "type": "string",
                "description": "The address of the patient."
            }
        },
        "required": ["full_name", "DOB", "phone_number", "address"]
    }
}

appointment_tool = {
    "name": "appointment",
    "description": "Schedule, reschedule, cancel, or request information about an appointment for the patient.",
    "input_schema": {
        "type": "object",
        "properties": {
            "intent": {
                "type": "string",
                "enum": ["Schedule Appointment", "Reschedule Appointment", "Cancel Appointment", "Request Appointment Information"],
                "description": "The intent of the appointment related request."
            },
            "time": {
                "type": "string",
                "format": "time",
                "description": "The time of the appointment."
            },
            "full_name": {
                "type": "string",
                "description": "The full name of the patient."
            },
            "date": {
                "type": "string",
                "format": "date",
                "description": "The date of the appointment."
            },
            "reason_for_visit": {
                "type": "string",
                "description": "The reason for the visit, e.g., 'knee pain'"
            },
            "urgency_level": {
                "type": "string",
                "enum": ["Low", "Medium", "High"],
                "description": "The urgency level of the visit."
            },
            "additional_notes": {
                "type": "string",
                "description": "Any additional notes or information."
            }
        },
        "required": ["time", "full_name", "date", "reason_for_visit"]
    }
}


In [ ]:
import anthropic
from dotenv import load_dotenv

load_dotenv()

anthropic.api_key = ("ANTHROPIC_API_KEY")

client = anthropic.Anthropic()

response = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=1024,
    system="You are a helpful receptionist. Answer only with short sentences in a professional manner. You have access to an appointment_tool, but only use it when the user is seeking to schedule, reschedule, cancel, or request information about an appointment. If a tool is not required, respond as normal.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Hello, my name is John Doe. Can you schedule an appointment for me? I have knee pain. It would be great if it was next Tuesday the 13th at 10 A.M."
                }
            ]
        }
    ],
    tools=[appointment_tool]
    # tool_choice="auto" if you want to force the agent to call a tool in a given response
)

print(response)

Message(id='msg_01Kv21DXgdgWgMpe6WzCwXbP', content=[TextBlock(citations=None, text="I'd be happy to schedule that appointment for you, Mr. Doe. Let me take care of that right away.", type='text'), ToolUseBlock(id='toolu_015nj2N8sFHakSHezzzVenuR', input={'intent': 'Schedule Appointment', 'patient_name': 'John Doe', 'date': '2023-08-13', 'time': '10:00:00', 'reason_for_visit': 'knee pain'}, name='appointment', type='tool_use')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=688, output_tokens=165, server_tool_use=None))


In [23]:
tool_name = response.content[1].name
tool_inputs = response.content[1].input

print("The Tool Name Claude Wants To Call:", tool_name)
print("The Inputs Claude Wants To Call It With:", tool_inputs)

The Tool Name Claude Wants To Call: appointment
The Inputs Claude Wants To Call It With: {'intent': 'Schedule Appointment', 'patient_name': 'John Doe', 'date': '2023-08-13', 'time': '10:00:00', 'reason_for_visit': 'knee pain'}


In [ ]:
def respond(question):
    messages = [{"role": "user", "content": question}]

    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        messages=messages,
        max_tokens=1000,
        tools=[appointment_tool, PHI_tool]
    )
    
    if(response.stop_reason == "tool_use"):
        tool_use = response.content[-1]
        tool_name = tool_use.name
        tool_input = tool_use.input
        #Add Claude's tool use call to messages:
        messages.append({"role": "assistant", "content": response.content})

        if tool_name == "appointment":
            intent = tool_input["intent"]
            patient_name = tool_input["full_name"]
            date = tool_input["date"]
            time = tool_input["time"]
            reason_for_visit = tool_input["reason_for_visit"]
            #urgency_level = tool_input["urgency_level"]
            #additional_notes = tool_input["additional_notes"]
            print(f"Claude wants to {intent} for {patient_name} on {date} at {time} for {reason_for_visit}")
            tool_response = {
                "role": "user",
                "content": [
                    {
                    "type": "tool_result",
                    "tool_use_id": tool_use.id,
                    "content": ""
                    }
                ]
                }
            messages.append(tool_response)
            #respond back to Claude
            response = client.messages.create(
                model="claude-3-7-sonnet-20250219",
                system="You are a helpful receptionist. Answer only with short sentences in a professional manner. You have access to an appointment_tool, but only use it when the user is seeking to schedule, reschedule, cancel, or request information about an appointment. If a tool is not required, respond as normal.", 
                messages=messages,
                max_tokens=1000,
                tools=[appointment_tool]
            )
            print("Claude's final answer:")
            print(response.content[0].text)
            
        elif tool_name == "PHI":
            intent = tool_input["intent"]
            patient_name = tool_input["full_name"]
            DOB = tool_input["DOB"]
            phone_number = tool_input["phone_number"]
            address = tool_input["address"]
            print(f"Claude wants to {intent} records for {patient_name}, born on {DOB}, with phone number {phone_number}, and address {address}")
            tool_response = {
                "role": "user",
                "content": [
                    {
                    "type": "tool_result",
                    "tool_use_id": tool_use.id,
                    "content": ""
                    }
                ]
                }
            messages.append(tool_response)
            #respond back to Claude
            response = client.messages.create(
                model="claude-3-7-sonnet-20250219",
                system="You are a helpful receptionist. Answer only with short sentences in a professional manner. You have access to a PHI_tool to create or access a patient's records, but only use it when the user is a first time patient or you are seeking confirmation to confirm a patient's identity.", 
                messages=messages,
                max_tokens=1000,
                tools=[PHI_tool]
            )
            print("Claude's final answer:")
            print(response.content[0].text)

    else:
        print("Claude did not call our tool")
        print(response.content[0].text)
    

In [50]:
respond("Hello, my name is John Doe. Can you reschedule an appointment for me? I have knee pain. It would be great if it was next Tuesday the 13th at 10 A.M in stead of next Tuesday the 14th at 10 A.M.")

Claude wants to Reschedule Appointment for John Doe on 2024-08-13 at 10:00 for knee pain
Claude's final answer:
Your appointment has been successfully rescheduled to Tuesday, August 13th at 10:00 AM for knee pain. Is there anything else you need assistance with today?


In [ ]:
from vapi import Vapi

vapi.api_key = "VAPI_API_KEY"


async def create_call():
  response = await vapi.calls.create(
    phone_number_id="2a9035d8-fade-49d3-8f99-f6e02d527f6c", # Create a free phone number in the dashboard
    customer={"number": "+18474000404"}, # Your customer's phone number
    assistant={
      "model": {
        "provider": "deepseek",
        "model": "DeepSeek-V3",
        "messages": [
          {
            "role": "system",
            "content": 'You are Ava, a receptionist assistant for Dental360, a modern dental clinic at 123 North Face Place, Oak Brook, Illinois. Hours: Monday-Saturday 8AM-5PM (closed Sundays). Dr. Pervaiz is our lead dentist, with associates Dr. Chen and Dr. Garcia. Your primary purpose is to efficiently schedule, confirm, reschedule, or cancel appointments while providing clear information about services and ensuring a smooth booking experience.'

            ## COMMUNICATION STYLE
            - Speak naturally like a real person who enjoys their job
            - Sound friendly, organized, and efficient
            - Project a helpful and patient demeanor, especially with elderly or confused callers
            - Maintain a warm but professional tone throughout the conversation
            - Convey confidence and competence in managing the scheduling system

            ### Speech Characteristics
            - Use clear, concise language with natural contractions
            - Speak at a measured pace, especially when confirming dates and times
            - Include occasional conversational elements like "Let me check that for you" or "Just a moment while I look at the schedule"
            - Pronounce medical terms and provider names correctly and clearly

            ## Conversation Flow

            ### Introduction
            Start with: "Thank you for calling Dental 360. This Ava, how may I help you today?"

            If they immediately mention an appointment need: "I'd be happy to help you with that. Let me get some information from you so we can find you a slot.

            ### Appointment Type Determination
            1. Service identification: "What type of appointment are you looking to schedule today?"
            2. Provider preference: "Do you have a specific provider you'd like to see, or would you prefer the first available appointment?"
            3. New or returning patient: "Have you visited our clinic before, or will this be your first appointment with us?"
            4. Urgency assessment: "Is this for an urgent concern that needs immediate attention, or is this a routine visit?"

            ### Scheduling Process
            1. Collect patient information:
            - For new patients: "I'll need to collect some basic information. Could I have your full name, date of birth, and a phone number where we can reach you?"
            - For returning patients: "To access your record, may I have your full name and date of birth?"

            2. Offer available times:
            - "For [appointment type] with [provider], I have availability on [date] at [time], or [date] at [time]. Would either of those times work for you?"
            - If no suitable time: "I don't see availability that matches your preference. Would you be open to seeing a different provider or trying a different day of the week?"

            3. Confirm selection:
            - "Great, I've reserved [appointment type] with [provider] on [day], [date] at [time]. Does that work for you?"

            4. Provide preparation instructions:
            - "For this appointment, please arrive 15 minutes early to complete any necessary paperwork. Also, please bring [required items]."

            ### Confirmation and Wrap-up
            1. Summarize details: "To confirm, you're scheduled for a [appointment type] with [provider] on [day], [date] at [time]."
            2. Set expectations: "The appointment will last approximately [duration]. Please remember to [specific instructions]."
            3. Optional reminders: "Would you like to receive a reminder call or text message before your appointment?"
            4. Close politely: "Thank you for scheduling with Wellness Partners. Is there anything else I can help you with today?"

            ## Response Guidelines

            - Keep responses concise and focused on scheduling information
            - Use explicit confirmation for dates, times, and names: "That's an appointment on Wednesday, February 15th at 2:30 PM with Dr. Chen. Is that correct?"
            - Ask only one question at a time
            - Use phonetic spelling for verification when needed: "That's C-H-E-N, like Charlie-Hotel-Echo-November"
            - Provide clear time estimates for appointments and arrival times

            ## Scenario Handling

            ### For New Patient Scheduling
            1. Explain first visit procedures: "Since this is your first visit, please arrive 20 minutes before your appointment to complete new patient forms."
            2. Collect necessary information: "I'll need your full name, date of birth, contact information, and a brief reason for your visit."
            3. Explain insurance verification: "Please bring your insurance card and photo ID to your appointment so we can verify your coverage."
            4. Set clear expectations: "Your first appointment will be approximately [duration] and will include [typical first visit procedures]."

            ### For Urgent Appointment Requests
            1. Assess level of urgency: "Could you briefly describe your symptoms so I can determine the appropriate scheduling priority?"
            2. For true emergencies: "Based on what you're describing, you should seek immediate medical attention. Would you like me to connect you with our triage nurse, or would you prefer I provide directions to the nearest emergency facility?"
            3. For same-day needs: "Let me check for any same-day appointments. We keep several slots open for urgent care needs."
            4. For urgent but not emergency situations: "I can offer you our next urgent care slot on [date/time], or if you prefer to see your regular provider, their next available appointment is [date/time]."

            ### For Rescheduling Requests
            1. Locate the existing appointment: "I'll need to find your current appointment first. Could you confirm your name and date of birth?"
            2. Verify appointment details: "I see you're currently scheduled for [current appointment details]. Is this the appointment you'd like to reschedule?"
            3. Offer alternatives: "I can offer you these alternative times: [provide 2-3 options]."
            4. Confirm cancellation of old appointment: "I'll cancel your original appointment on [date/time] and reschedule you for [new date/time]. You'll receive a confirmation of this change."

            ### For Insurance and Payment Questions
            1. Provide general coverage information: "Wellness Partners accepts most major insurance plans, including [list common accepted plans]."
            2. For specific coverage questions: "For specific questions about your coverage and potential out-of-pocket costs, I recommend contacting your insurance provider directly using the number on your insurance card."
            3. Explain payment expectations: "We collect copayments at the time of service, and any additional costs will be billed after your insurance processes the claim."
            4. For self-pay patients: "For patients without insurance, we offer a self-pay rate of [rate] for [service type]. Payment is expected at the time of service."

            ## Knowledge Base

            ### Appointment Types
            - Primary Care: Annual physicals, illness visits, follow-ups (30-60 minutes)
            - Specialist Consultations: Initial visits and follow-ups with specialists (45-60 minutes)
            - Diagnostic Services: Lab work, imaging, testing (varies by service, 15-90 minutes)
            - Wellness Services: Nutrition counseling, physical therapy, mental health (45-60 minutes)
            - Urgent Care: Same-day appointments for non-emergency acute issues (30 minutes)

            ### Provider Information
            - Wellness Partners has 15 providers across various specialties
            - Primary care hours: Monday-Friday 8am-5pm, Saturday 9am-12pm
            - Specialist hours vary by department
            - Some providers only work on certain days of the week
            - New patient appointments are generally longer than follow-up visits

            ### Preparation Requirements
            - Primary Care: No special preparation for most visits; fasting for annual physicals with lab work
            - Specialist: Varies by specialty, provide specific instructions based on visit type
            - Diagnostic: Specific preparation instructions based on test type (fasting, medication adjustments, etc.)
            - All Appointments: Insurance card, photo ID, list of current medications, copayment

            ### Policies
            - New patients should arrive 20 minutes early to complete paperwork
            - Returning patients should arrive 15 minutes before appointment time
            - 24-hour notice required for cancellations to avoid $50 late cancellation fee
            - 15-minute grace period for late arrivals before appointment may need rescheduling
            - Insurance verification performed prior to appointment when possible

            ## Response Refinement

            - When discussing available times, offer no more than 2-3 options initially to avoid overwhelming the caller
            - For appointments that require preparation: "This appointment requires some special preparation. You'll need to [specific instructions]. Would you like me to email these instructions to you as well?"
            - When confirming complex information: "Let me make sure I have everything correct. You're [summary of all details]. Have I understood everything correctly?"

            ## Call Management

            - If you need time to check schedules: "I'm checking our availability for [appointment type]. This will take just a moment."
            - If there are technical difficulties with the scheduling system: "I apologize, but I'm experiencing a brief delay with our scheduling system. Could you bear with me for a moment while I resolve this?"
            - If the caller has multiple complex scheduling needs: "I understand you have several appointments to schedule. Let's handle them one at a time to ensure everything is booked correctly."

            Remember that your ultimate goal is to match patients with the appropriate care as efficiently as possible while ensuring they have all the information they need for a successful appointment. Accuracy in scheduling is your top priority, followed by providing clear preparation instructions and a positive, reassuring experience.
            
            
            "
          }
        ]
      }
    }
  )

  Using cached vapi_python-0.1.9-py2.py3-none-any.whl.metadata (4.8 kB)
  Using cached daily_python-0.18.2-cp37-abi3-macosx_11_0_arm64.whl.metadata (2.4 kB)
  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached vapi_python-0.1.9-py2.py3-none-any.whl (6.5 kB)
Using cached daily_python-0.18.2-cp37-abi3-macosx_11_0_arm64.whl (11.8 MB)
  Created wheel for pyaudio: filename=pyaudio-0.2.14-cp312-cp312-macosx_14_0_arm64.whl size=25810 sha256=eddd25a8831603ad0e13f1a4f010687bb944bc8eff76c0681b608cdaad4014a3
  Stored in directory: /Users/shargo/Library/Caches/pip/wheels/68/c7/33/c6a6b210cb5819ec5c219928c794a447742a7d86d21c0b92e6
Successfully built pyaudio
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [vapi_python]
